<a href="https://colab.research.google.com/github/saat12d/NaturalLanguageProcessing/blob/main/deep_learning_for_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing data (imdb reviews)

In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  39.5M      0  0:00:02  0:00:02 --:--:-- 39.5M


Delete unrequired directory

In [ ]:
!rm -r aclImdb/train/unsup

Inspect data

In [ ]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

Prepare validation set (20% of training set)

In [ ]:
import os, pathlib, shutil, random

base_dir = pathlib.Path('aclImdb')
val_dir = base_dir / 'val'
train_dir = base_dir / 'train'

for category in ('neg', 'pos'):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.Random(1337).shuffle(files)
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    shutil.move(train_dir / category / fname, val_dir / category / fname)

Prepare datasets for training, validation, testing

In [ ]:
from tensorflow import keras
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory('aclImdb/train', batch_size = batch_size)
val_ds = keras.utils.text_dataset_from_directory('aclImdb/val', batch_size = batch_size)
test_ds = keras.utils.text_dataset_from_directory('aclImdb/test', batch_size = batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


# Bag of Words model

## Unigrams

Processing Dataset with a TextVectorization Layer

In [ ]:
from tensorflow.keras.layers import TextVectorization
text_vectorization = TextVectorization(
    max_tokens = 20000,
    output_mode = 'multi_hot'
)
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)

binary_1gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls = 4)
binary_1gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls = 4)
binary_1gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls = 4)

Reusable Model Building Function

In [ ]:
from tensorflow.keras import layers

def get_model(max_tokens = 20000, hidden_dims = 16):
  inputs = keras.Input(shape = (max_tokens,))
  x = layers.Dense(hidden_dims, activation = 'relu')(inputs)
  x = layers.Dropout(0.5)(x)
  outputs = layers.Dense(1, activation = 'sigmoid')(x)
  model = keras.Model(inputs, outputs)
  model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model

Training and testing binary unigram model

In [ ]:
model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint(filepath = 'binary_1gram.keras', save_best_only = True)
]

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# We call cache() on the datasets to cache them in memory: this way, we will only do the preprocessing once, during the first epoch, and we’ll reuse the preprocessed texts for the following epochs. This can only be done if the data
# is small enough to fit in memory
model.fit(binary_1gram_train_ds.cache(), validation_data = binary_1gram_val_ds.cache(), epochs = 10, callbacks = callbacks)

Epoch 1/10
625/625 [==============================] - 10s 14ms/step - loss: 0.4084 - accuracy: 0.8249 - val_loss: 0.2829 - val_accuracy: 0.8906
Epoch 2/10
625/625 [==============================] - 5s 8ms/step - loss: 0.2728 - accuracy: 0.8980 - val_loss: 0.2779 - val_accuracy: 0.8950
Epoch 3/10
625/625 [==============================] - 4s 7ms/step - loss: 0.2363 - accuracy: 0.9177 - val_loss: 0.2871 - val_accuracy: 0.8994
Epoch 4/10
625/625 [==============================] - 5s 8ms/step - loss: 0.2266 - accuracy: 0.9240 - val_loss: 0.3025 - val_accuracy: 0.8958
Epoch 5/10
625/625 [==============================] - 5s 9ms/step - loss: 0.2157 - accuracy: 0.9300 - val_loss: 0.3342 - val_accuracy: 0.8900
Epoch 6/10
625/625 [==============================] - 5s 8ms/step - loss: 0.2110 - accuracy: 0.9340 - val_loss: 0.3351 - val_accuracy: 0.8926
Epoch 7/10
625/625 [==============================] - 5s 8ms/step - loss: 0.2009 - accuracy: 0.9366 - val_loss: 0.3454 - val_accuracy: 0.8906
Epoc

In [ ]:
model = keras.models.load_model('binary_1gram.keras')
print(f"Test Accuracy: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

782/782 [==============================] - 4s 5ms/step - loss: 0.2889 - accuracy: 0.8872
Test Accuracy: 0.887


## Bigrams

In [ ]:
text_vectorization = TextVectorization(
    ngrams = 2,
    max_tokens = 20000,
    output_mode = 'multi_hot'
)

Training and testing the binary 2gram model

In [ ]:
text_vectorization.adapt(text_only_train_ds)
binary_2gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls = 4)
binary_2gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls = 4)
binary_2gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls = 4)

model = get_model()
callbacks = keras.callbacks.ModelCheckpoint(filepath = 'binary_2gram.keras', save_best_only = True)

model.fit(binary_2gram_train_ds.cache(), validation_data = binary_2gram_val_ds.cache(), epochs = 10, callbacks = callbacks)

model = keras.models.load_model('binary_2gram.keras')
print(f"Test Accuracy: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

Epoch 1/10
625/625 [==============================] - 10s 14ms/step - loss: 0.3902 - accuracy: 0.8347 - val_loss: 0.2672 - val_accuracy: 0.8960
Epoch 2/10
625/625 [==============================] - 6s 9ms/step - loss: 0.2466 - accuracy: 0.9096 - val_loss: 0.2571 - val_accuracy: 0.9022
Epoch 3/10
625/625 [==============================] - 6s 9ms/step - loss: 0.2115 - accuracy: 0.9283 - val_loss: 0.2720 - val_accuracy: 0.9028
Epoch 4/10
625/625 [==============================] - 4s 6ms/step - loss: 0.1913 - accuracy: 0.9389 - val_loss: 0.3014 - val_accuracy: 0.8996
Epoch 5/10
625/625 [==============================] - 5s 9ms/step - loss: 0.1820 - accuracy: 0.9463 - val_loss: 0.3033 - val_accuracy: 0.9028
Epoch 6/10
625/625 [==============================] - 5s 8ms/step - loss: 0.1741 - accuracy: 0.9490 - val_loss: 0.3169 - val_accuracy: 0.9000
Epoch 7/10
625/625 [==============================] - 5s 8ms/step - loss: 0.1713 - accuracy: 0.9519 - val_loss: 0.3334 - val_accuracy: 0.8976
Epoc

## Bigrams with TF-IDF encoding

In [ ]:
text_vectorization = TextVectorization(
    ngrams = 2,
    max_tokens = 20000,
    output_mode = 'tf_idf'
)

In [ ]:
text_vectorization.adapt(text_only_train_ds)
tfidf_2gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls = 4)
tfidf_2gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls = 4)
tfidf_2gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls = 4)

model = get_model()
callbacks = keras.callbacks.ModelCheckpoint(filepath = 'tfidf_2gram.keras', save_best_only = True)

model.fit(tfidf_2gram_train_ds.cache(), validation_data = tfidf_2gram_val_ds.cache(), epochs = 10, callbacks = callbacks)

model = keras.models.load_model('tfidf_2gram.keras')
print(f"Test Accuracy: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

Epoch 1/10
625/625 [==============================] - 12s 18ms/step - loss: 0.5375 - accuracy: 0.7684 - val_loss: 0.2901 - val_accuracy: 0.8890
Epoch 2/10
625/625 [==============================] - 5s 7ms/step - loss: 0.3518 - accuracy: 0.8511 - val_loss: 0.2940 - val_accuracy: 0.8844
Epoch 3/10
625/625 [==============================] - 5s 8ms/step - loss: 0.3176 - accuracy: 0.8697 - val_loss: 0.3086 - val_accuracy: 0.8812
Epoch 4/10
625/625 [==============================] - 4s 7ms/step - loss: 0.2971 - accuracy: 0.8839 - val_loss: 0.3151 - val_accuracy: 0.8752
Epoch 5/10
625/625 [==============================] - 5s 8ms/step - loss: 0.2771 - accuracy: 0.8888 - val_loss: 0.3080 - val_accuracy: 0.8806
Epoch 6/10
625/625 [==============================] - 4s 7ms/step - loss: 0.2698 - accuracy: 0.8959 - val_loss: 0.3471 - val_accuracy: 0.8838
Epoch 7/10
625/625 [==============================] - 4s 7ms/step - loss: 0.2591 - accuracy: 0.8981 - val_loss: 0.3291 - val_accuracy: 0.8776
Epoc

# Sequence Models

In [ ]:
max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)
int_train_ds = train_ds.map(
lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_val_ds = val_ds.map(
lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_test_ds = test_ds.map(
lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

## Simple model with bidirectional LSTM

In [ ]:
import tensorflow as tf
inputs = keras.Input(shape = (None,), dtype = 'int64')
embedded = tf.one_hot(inputs, depth = max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model = keras.Model(inputs, outputs)
model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 20000)       0         
                                                                 
 bidirectional (Bidirectiona  (None, 64)               5128448   
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 5,128,513
Trainable params: 5,128,513
Non-trainable params: 0
___________________________________________________

In [ ]:
callbacks = keras.callbacks.ModelCheckpoint('one_hot_bidir_lstm.keras', save_best_only = True)
model.fit(int_train_ds, validation_data = int_val_ds, epochs = 10, callbacks = callbacks)
print(f"Test Accuracy: {model.evaluate(int_test_ds)[1]:.3f}")

NameError: ignored

## Model with Embedding Layer trained from scratch

In [ ]:
inputs = keras.Input(shape = (None,), dtype = 'int64')
embedded = layers.Embedding(input_dim = max_tokens, output_dim = 256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model = keras.Model(inputs, outputs)

model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])
model.summary()

callbacks = keras.callbacks.ModelCheckpoint('embeddings_bidir_lstm.keras', save_best_only = True)
model.fit(int_train_ds, validation_data = int_val_ds, epochs = 10, callbacks = callbacks)
print(f"test accuracy: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_3 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_3 (Bidirectio  (None, 64)               73984     
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________

## Model with Embedding layer and masking enabled

In [ ]:
inputs = keras.Input(shape = (None,), dtype = 'int64')
embedded = layers.Embedding(input_dim = max_tokens, output_dim = 256, mask_zero = True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model = keras.Model(inputs, outputs)

model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])
model.summary()

callbacks = keras.callbacks.ModelCheckpoint('embeddings_bidir_lstm_with_masking.keras', save_best_only = True)
model.fit(int_train_ds, validation_data = int_val_ds, epochs = 10, callbacks = callbacks)
print(f"test accuracy: {model.evaluate(int_test_ds)[1]:.3f}")

## Using pretrained word embeddings

Import GloVe word embeddings

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2023-07-29 05:24:36--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-07-29 05:24:36--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-07-29 05:24:37--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

Parsing the word embeddings file

In [ ]:
import numpy as np
path_to_glove_file = 'glove.6B.100d.txt'

embeddings_index = {}
with open(path_to_glove_file) as f:
  for line in f:
    word, coefs = line.split(maxsplit = 1)
    coefs = np.fromstring(coefs, 'f', sep = ' ')
    embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors")

Found 400000 word vectors


Preparing the word embeddings matrix

In [ ]:
embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
  if i < max_tokens:
    embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector



In [ ]:
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer = keras.initializers.Constant(embedding_matrix),
    trainable = False,
    mask_zero = True
)

Model

In [ ]:
inputs = keras.Input(shape = (None,), dtype = 'int64')
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model = keras.Model(inputs, outputs)

model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])
model.summary()

callbacks = keras.callbacks.ModelCheckpoint('glove_embeddings_sequence_model.keras', save_best_only = True)
model.fit(int_train_ds, validation_data = int_val_ds, epochs = 10, callbacks = callbacks)
print(f"test accuracy: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         2000000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               34048     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,034,113
Trainable params: 34,113
Non-trainable params: 2,000,000
____________________________________________

# Transformer Architecture

##Transformer Encoder for text classification

Transformer encoder implemented as a subclassed layer

In [ ]:
class TransformerEncoder(layers.Layer):
  def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim
    self.dense_dim = dense_dim
    self.num_heads = num_heads
    self.attention = layers.MultiHeadAttention(num_heads = num_heads, key_dim = embed_dim)
    self.dense_proj = keras.Sequential([layers.Dense(dense_dim, activation = 'relu'), layers.Dense(embed_dim)])
    self.layernorm_1 = layers.LayerNormalization()
    self.layernorm_2 = layers.LayerNormalization()

  def call(self, inputs, mask = None):
    if mask is not None:
      mask = mask[:, tf.newaxis, :]
    attention_output = self.attention(inputs, inputs, attention_mask = mask)
    proj_input = self.layernorm_1(inputs + attention_output)
    proj_output = self.dense_proj(proj_input)
    return self.layernorm_2(proj_input + proj_output)

  def get_config(self):
    config = super().get_config()
    config.update({
        "embed_dim": self.embed_dim,
        "num_heads": self.num_heads,
        "dense_dim": self.dense_dim
    })
    return config

In [ ]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape = (None,), dtype = 'int64')
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         5120000   
                                                                 
 transformer_encoder (Transf  (None, None, 256)        543776    
 ormerEncoder)                                                   
                                                                 
 global_max_pooling1d (Globa  (None, 256)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257   

In [ ]:
callbacks = keras.callbacks.ModelCheckpoint('transformer_encoder.keras', save_best_only = True)
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)

Epoch 1/20
313/625 [==============>...............] - ETA: 20:52 - loss: 0.5779 - accuracy: 0.7285

KeyboardInterrupt: ignored

In [ ]:
model = keras.models.load_model(
    "transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

## Positional Embedding (to inject word order information)

In [ ]:
class PositionalEmbedding(layers.Layer):

  def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
    super().__init__(**kwargs)
    self.token_embeddings = layers.Embedding(input_dim = input_dim, output_dim = output_dim)
    self.position_embeddings = layers.Embedding(input_dim = sequence_length, output_dim = output_dim)
    self.sequence_length = sequence_length
    self.input_dim = input_dim
    self.output_dim = output_dim

  def call(self, inputs):
    length = tf.shape(inputs)[-1]
    positions = tf.range(start = 0, limit = length, delta = 1)
    embedded_tokens = self.token_embeddings(inputs)
    embedded_positions = self.position_embeddings(positions)
    return embedded_tokens + embedded_positions

  def get_config(self):
    config = super().get_config()
    config.update({
        "output_dim": self.output_dim,
        "sequence_length": self.sequence_length,
        "input_dim": self.input_dim
    })
    return config

In [ ]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20,
     callbacks=callbacks)


In [ ]:
model = keras.models.load_model(
    "full_transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
"PositionalEmbedding": PositionalEmbedding})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")